# Generative Adversarial Networks

We’ve talked about how to make predictions. In some form or another, we used deep neural networks learned mappings from data points to labels. This kind of learning is called discriminative learning, as in, we’d like to be able to discriminate between photos cats and photos of dogs. Classifiers and regressors are both examples of discriminative learning. And neural networks trained by backpropagation have upended everything we thought we knew about discriminative learning on large complicated datasets. Classification accuracies on high-res images has gone from useless to human-level (withsome caveats) in just 5-6 years. We’ll spare you another spiel about all the other discriminative tasks where deep neural networks do astoundingly well.

But there’s more to machine learning than just solving discriminative tasks. For example, given a largedataset, without any labels, we might want to learn a model that concisely captures the characteristics of this data. Given such a model, we could sample synthetic data points that resemble the distribution ofthe training data. For example, given a large corpus of photographs of faces, we might want to be able togenerate a new photorealistic image that looks like it might plausibly have come from the same dataset.This kind of learning is called `generative modeling`.

Until recently, we had no method that could synthesize novel photorealistic images. But the success of deep neural networks for discriminative learning opened up new possibilities. One big trend over the last three years has been the application of discriminative deep nets to overcome challenges in problems that we don’t generally think of as supervised learning problems. The recurrent neural network language models are one example of using a discriminative network (trained to predict the next character) that once trained can actas a generative model.

In 2014, a breakthrough paper introduced `Generative adversarial networks (GANs)`, a clever new way to leverage the power of discriminative models to get good generative models. At their heart, GANs rely on the idea that a data generator is good if we cannot tell fake data apart from real data. In statistics, this is calleda two-sample test - a test to answer the question whether datasets $X= {x_1;...;x_n}$ and $X'= {x'_1;...;x'_n}$ were drawn from the same distribution. The main difference between most statistics papers and GANs is that the latter use this idea in a constructive way. In other words, rather than just training a model to say “hey, these two datasets don’t look like they came from the same distribution”, they use the `two-sample test` to provide training signal to a generative model. This allows us to improve the data generator until it generates something that resembles the real data. At the very least, it needs to fool the classifier. And ifour classifier is a state of the art deep neural network.

The GANs architecture is illustrated in `Figure 1`. As you can see, there are two pieces to GANs - first off, we need a device (say, a deep network but it really could be anything, such as a game rendering engine) that might potentially be able to generate data that looks just like the real thing. If we are dealing with images, this needs to generate images. If we’re dealing with speech, it needs to generate audio sequences, and soon. We call this the generator network. The second component is the discriminator network. It attempts to distinguish fake and real data from each other. Both networks are in competition with each other. The generator network attempts to fool the discriminator network. At that point, the discriminator network adapts to the new fake data. This information, in turn is used to improve the generator network, and so on.

<p align="center">
    <img src="images/gan.svg"><br>
    Figure 1. Generative Adversarial Networks
</p>

The discriminator is a binary classifier to distinguish if the inputxis real (from real data) or fake (from the generator). Typically, the discriminator outputs a scalar prediction $ o \in R $ for input $\mathbf x$, such as using a dense layer with hidden size 1, and then applies sigmoid function to obtain the predicted probability $ D(x) = 1/(1 + e^{-x}) $. Assume the label y for true data is 1 and 0 for fake data. We train the discriminator to minimize the cross entropy loss, i.e $$ \min - y \log D(\mathbf x) - (1-y)\log(1-D(\mathbf x)) $$

For the generator, it first draws some parameter $ z \in R^d $ from a source of randomness, e.g. a normal distribution $ \mathbf z \sim\mathcal(0;1) $. We often call z the `latent variable`. It then applies a function to generate $ \mathbf x′=G(\mathbf z) $. The goal of the generator is to fool the discriminator to classify $x′$ as true data. In other words, we update the parameters of the generator to maximize the cross entropy loss when y= 0, i.e. $$ \max - \log(1-D(\mathbf x'))$$

If the discriminator does a perfect job, then $ D(\mathbf x')\approx 1 $ so the above loss near 0, which results the gradients are too small to make a good progress for the discriminator. So commonly we minimize the following loss $$ \max \log(D(\mathbf x')) $$

which is just feed $\mathbf x' $ into the discriminator but giving label $y=1$.

Many of the GANs applications are in the context of images. As a demonstration purpose, we're going to content ourselves with fitting a much simpler distribution first. We will illustrate what happens if we use GANs to build the world's most inefficient estimator of parameters for a Gaussian. Let's get started.

## 1. Import necessary dependencies
-----------------------------------

In [ ]:
%matplotlib inline

import d2l
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import Dataset
from torch.utils.data import DataLoader

## 2. Generate some “real” data
-------------------------------

Since this is going to be the world’s lamest example, we simply generate data drawn from a Gaussian.

In [ ]:
X = np.random.normal(size=(1000, 2))
X = torch.from_numpy(X).float()
A = torch.tensor([[ 1. , 2. ],
                  [-0.1, 0.5]])
b = torch.tensor([1., 2.])
data = torch.matmul(X, A) + b


class GANDataset(Dataset):
    
    def __init__(self, data, transforms=None):
        self.dataset = data
    
    def __len__(self):
        return self.dataset.size(0)
    
    def __getitem__(self, idx):
        return self.dataset[idx]

In [ ]:
plt.scatter(data[:100, 0].numpy(), data[:100,  1].numpy())
plt.show()
print(f'The covariance matrix is {torch.matmul(A.T,A)}')

## 3. Generator
---------------

Our generator network will be the simplest network possible - a single layer linear model. This is since we’llbe driving that linear network with a Gaussian data generator. Hence, it literally only needs to learn the parameters to fake things perfectly.

In [ ]:
def get_generator(device=None):
    net_G = nn.Sequential(
        nn.Linear(2, 2)
    )
    
    if device is not None:
        net_G.to(device)
    
    return net_G

## 4. Discriminator
-------------------

For the discriminator we will be a bit more discriminating: we will use an MLP with 3 layers to make things a bit more interesting.

In [ ]:
def get_discriminator(device=None):
    net_D = nn.Sequential(
        nn.Linear(2, 5),
        nn.Tanh(),
        nn.Linear(5, 3),
        nn.Tanh(),
        nn.Linear(3, 1)
    )
    
    if device is not None:
        net_D.to(device)
    
    return net_D

## 5. Training
--------------

First we define a function to update the discriminator.

In [ ]:
def update_D(X, Z, net_D, net_G, loss, optimizer_D):
    """Updates discriminator."""
    batch_size = X.size(0)
    ones = X.new_ones(size=(batch_size, 1))
    zeros = X.new_zeros(size=(batch_size, 1))
    
    # zero the parameter gradients
    optimizer_D.zero_grad()
    
    real_Y = net_D(X)
    fake_X = net_G(Z)
    # Don't need to compute gradient for net_G, detach it from
    # computing gradients.
    fake_Y = net_D(fake_X.detach())
    loss_D = (loss(real_Y, ones) + loss(fake_Y, zeros)) / 2
    
    loss_D.backward()
    optimizer_D.step()
    
    return loss_D.sum().item()

The generator is updated similarly. Here we reuse the cross-entropy loss but change the label of the fake data from $0$ to $1$.

In [ ]:
def update_G(Z, net_D, net_G, loss, optimizer_G):
    """Updates generator."""
    batch_size = Z.size(0)
    ones = Z.new_ones(size=(batch_size, 1))
    
    # zero the parameter gradients
    optimizer_G.zero_grad()
    
    # We could reuse fake_X from update_D to save computation.
    fake_X = net_G(Z)
    # Recomputing fake_Y is needed since net_D is changed.
    fake_Y = net_D(fake_X)
    loss_G = loss(fake_Y, ones)
    
    loss_G.backward()
    optimizer_G.step()
    
    return loss_G.sum().item()

Both the discriminator and the generator performs a binary logistic regression with the cross-entropy loss. We use Adam to smooth the training process. In each iteration, we first update the discriminator and then the generator. We visualize both losses and generated examples.

In [ ]:
def train(net_D, net_G, data_iter, num_epochs, lr_D, lr_G, latent_dim, data):
    loss = nn.BCEWithLogitsLoss()
    optimizer_D = optim.Adam(net_D.parameters(), lr=lr_D)
    optimizer_G = optim.Adam(net_G.parameters(), lr=lr_G)
    
    animator = d2l.Animator(xlabel='epoch', ylabel='loss',
                            xlim=[1, num_epochs], nrows=2, figsize=(5,5),
                            legend=['generator', 'discriminator'])
    animator.fig.subplots_adjust(hspace=0.3)
    
    for epoch in range(num_epochs):
        # Train one epoch
        timer = d2l.Timer()
        metric = d2l.Accumulator(3)  # loss_D, loss_G, num_examples
        
        for _, X in enumerate(data_iter):
            batch_size = X.size(0)
            X = X.view(-1, latent_dim)
            X = X.to(device)
            
            # z latent variables
            Z = np.random.normal(0, 1, size=(batch_size, latent_dim))
            Z = torch.from_numpy(Z).float()
            Z = Z.view(-1, latent_dim)
            Z = Z.to(device)
            
            metric.add(update_D(X, Z, net_D, net_G, loss, optimizer_D),
                       update_G(Z, net_D, net_G, loss, optimizer_G),
                       batch_size)
        
        # Visualize generated examples
        Z = np.random.normal(0, 1, size=(100, latent_dim))
        Z = torch.from_numpy(Z).float().to(device)
        with torch.no_grad():
            fake_X = net_G(Z).cpu()
            fake_X = fake_X.numpy()
        animator.axes[1].cla()
        animator.axes[1].scatter(data[:,0], data[:,1])
        animator.axes[1].scatter(fake_X[:,0], fake_X[:,1])
        animator.axes[1].legend(['real', 'generated'])
        
        # Show the losses
        loss_D, loss_G = metric[0]/metric[2], metric[1]/metric[2]
        animator.add(epoch, (loss_D, loss_G))
    
    print(f'loss_D {loss_D:.3f}, loss_G {loss_G:.3f}, {metric[2]/timer.stop()} examples/sec')

Now we specify the hyper-parameters to fit the Gaussian distribution.

In [ ]:
lr_D = 0.05
lr_G = 0.005
latent_dim = 2
num_epochs = 20

trainset = GANDataset(data)
trainloader = DataLoader(trainset, batch_size=2, shuffle=True, num_workers=4)

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

net_G = get_generator(device)
net_D = get_discriminator(device)
train(net_D, net_G, trainloader,
      num_epochs, lr_D, lr_G,
      latent_dim, data[:100].numpy())

## Summary
----------

- Generative adversarial networks (GANs) composes of two deep networks, the generator and the discriminator.
- The generator generates the image as much closer to the true image as possible to fool the discriminator, via maximizing the cross-entropy loss, i.e., $\max \log(D(\mathbf{x'}))$.
- The discriminator tries to distinguish the generated images from the true images, via minimizing the cross-entropy loss, i.e., $\min - y \log D(\mathbf{x}) - (1-y)\log(1-D(\mathbf{x}))$.